In [2]:
import pandas as pd
from collections.abc import Iterable
data = pd.read_csv("kor_grads_data.csv")
data.head()

,학제,school_name,school_id,학교상태,본분교,시도,설립,과정구분,대계열,major_department,...,3차 유지취업률_여,4차 유지취업자_계,4차 유지취업자_남,4차 유지취업자_여,4차 유지취업률_계,4차 유지취업률_남,4차 유지취업률_여,입학당시 기취업자_계,입학당시 기취업자_남,입학당시 기취업자_여
0,전문대학,가톨릭상지대학교,73001000,기존,본교,경북,사립,전문대학과정,사회계열,경영ㆍ경제,...,100.0,2,1,1,66.7,50.0,100.0,2,2,0
1,전문대학,가톨릭상지대학교,73001000,기존,본교,경북,사립,전문대학과정,사회계열,사회과학,...,0.0,4,4,0,80.0,80.0,0.0,1,1,0
2,전문대학,가톨릭상지대학교,73001000,기존,본교,경북,사립,전문대학과정,교육계열,유아교육,...,82.4,14,0,14,82.4,0.0,82.4,16,0,16
3,전문대학,가톨릭상지대학교,73001000,기존,본교,경북,사립,전문대학과정,공학계열,기계ㆍ금속,...,0.0,3,3,0,100.0,100.0,0.0,0,0,0
4,전문대학,가톨릭상지대학교,73001000,기존,본교,경북,사립,전문대학과정,공학계열,기계ㆍ금속,...,0.0,6,6,0,85.7,85.7,0.0,2,2,0


##### Introduction.
Students in Korea dedicate their lives in order to gain admission into the most prestigious schools in the country. Presumably this is all in hopes of securing a well-paying, job. Through this analysis I wanted to test the validity of this hope. However, I found that Korean universities are very bad at publishing data about their alumni -- American universities on the other hand, are great at this. Thus, I resorted to analyzing correlations between various factors related to majors at specific universities and employment rates. 

The main variables I was interested in were the name of the school (school_name/school_id), the major (major_department/major_id), and the employment rate (total_employment_rate). 

In [3]:
#takes string ids as input and spits out integer conversion
def convert_string_id(id):
    if not isinstance(id,Iterable): return id
    for c in id: 
        id = id.replace(c,str(ord(c)))
    return int(id)

data = data.applymap(convert_string_id)


/var/folders/y0/sc7jmx7n2vs9kmrjt2zyfpk00000gn/T/ipykernel_80512/1926497852.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(convert_string_id)


In [4]:
def get_corr(var1,var2):
    return data[var1].corr(data[var2])

def find_largest_corr(data,num_top,outcome_var):
    top_corrs = list(map(lambda col: data.columns[col],range(num_top)))
    comp_corr = lambda x: get_corr(x,outcome_var)
    for i in range(num_top,data.shape[1]):
        if data.columns[i] == outcome_var: pass
        elif min(map(comp_corr,top_corrs)) < get_corr(data.columns[i],outcome_var):
            top_corrs[top_corrs.index(min(top_corrs,key=comp_corr))] = data.columns[i]
        top_corrs.sort(key=comp_corr,reverse=True)
    return top_corrs

def find_largest_corr(data, num_top, outcome_var):
    top_corrs = {}
    for i in range(num_top):
        top_corrs[data.columns[i]] = get_corr(data.columns[i], outcome_var)
    comp_corr = lambda x: get_corr(x,outcome_var)
    for i in range(num_top, data.shape[1]):
        col_name = data.columns[i]
        curr_corr = comp_corr(col_name)
        if col_name == outcome_var: pass
        elif min(top_corrs.values()) < curr_corr:
            min_corr_col = min(top_corrs, key=top_corrs.get)
            top_corrs.pop(min(top_corrs,key=comp_corr))
            top_corrs[col_name] = curr_corr
    sorted_top_corrs = dict(sorted(top_corrs.items(), key=lambda item: item[1], reverse=True))
    return sorted_top_corrs


In [9]:
data_exclude = data.iloc[:,:data.columns.get_loc("men_employment_rate")]
find_largest_corr(data_exclude,10,"total_employment_rate")

{'과정구분': np.float64(0.05690551651171578),
 '졸업자_남': np.float64(0.020906452884062326),
 '졸업자_계': np.float64(0.01644724545259603),
 '학교상태': np.float64(0.013568427430676825),
 '설립': np.float64(0.011216372427720398),
 'school_name': np.float64(0.007675138564452765),
 'major_id': np.float64(0.0076692537372907875),
 'school_id': np.float64(0.006922570687511582),
 '본분교': np.float64(0.006438908014242989),
 '졸업자_여': np.float64(0.005783177492783677)}

The variable that has the highest correlation with employment rates was the type of degree program, i.e. four year degree, two year degree, graduate school degree, etc. However, even this variable has a very low correlation with the employment rate, around 5%.

There aren't that many sweeping claims that can be made from this relatively restricted data set. With no information about salaries or the types of jobs that these people are getting, it's hard to say whether there is or isn't a correlation between one's university name and one's job.